# SCTK Defect Supercell Testing

In [2]:
from ase import Atoms
from ase.build import bulk, make_supercell
import numpy as np


## Evaluate Defect Site placement in large lattices:

In [3]:
from ase.visualize import view
atoms = bulk('C', 'sc', a=3.0)

In [4]:
from sctk.structure import DefectSupercellBuilder

dsb = DefectSupercellBuilder(atoms)
dsb.set_supercell_matrix(4*np.eye(3))
dsb.add_substitution(elem='C', sub='B', n=8, loc='auto')
dsb.add_interstitial(elem='K', n=1, loc='center')

example_sc = dsb.get(random=False)

In [5]:
view(example_sc, viewer='x3d')

In [6]:
example_sc.write('./figures2/doping_example.poscar')

## Evaluate Doping to match material

In [7]:
from sctk.databases import MaterialsProject
from sctk.materials import Material
from sctk.structure import StructurePhaseDiagram

material_str = 'Cs3 C60 K x'
pristine_material = Material(material_str).get_pristine_material()

print('material:')
display(Material(material_str))
print('pristine:')
display(pristine_material)

material:


pristine:


In [8]:
spd = StructurePhaseDiagram(pristine_material)

with MaterialsProject() as mp:
    spd.add_database_structures(mp)

_, pristine_struct = spd.find_closest_structure(pristine_material, max_ehull=1.0)

/media/colin/Shared/colin/git/materials-ml/env/lib/python3.10/site-packages/mp_api/client/mprester.py:146: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
  0%|                                                     | 0/3 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/15 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/15 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/15 [00:00<?, ?it/s]

 33%|███████████████                              | 1/3 [00:01<00:02,  1.20s/it]

...

 67%|██████████████████████████████               | 2/3 [00:02<00:01,  1.36s/it]

Retrieving SummaryDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 3/3 [00:04<00:00,  1.44s/it]


In [9]:
pristine_material = Material(pristine_struct.get_chemical_formula())
display(pristine_material)

In [10]:
dsb = DefectSupercellBuilder(pristine_struct)
material_sub = Material(material_str).substitute({'x' : 1})
dsb.dope_like_material(material_sub,'ppd', n_unitcells=4, centering=False)
supercell = dsb.get(random=False)

In [11]:
view(supercell, viewer='x3d')

In [12]:
supercell.write(f'./figures2/{pristine_material.str(pretty=False)}.cif')
supercell.write(f'./figures2/{material_sub.str(pretty=False)}.cif')